### Merge Title and Comments

In [1]:
import pandas as pd
from tqdm import tqdm

SIZE = "small"

In [2]:
train_data = pd.read_csv("../data/text_metadata/multimodal_only_samples/multimodal_train.tsv", sep='\t')
test_data = pd.read_csv("../data/text_metadata/multimodal_only_samples/multimodal_test_public.tsv", sep='\t')

In [3]:
train_data_selected = train_data[["clean_title", "id", "image_url", "2_way_label", "3_way_label", "6_way_label"]]
test_data_selected = test_data[["clean_title", "id", "image_url", "2_way_label", "3_way_label", "6_way_label"]]

In [4]:
TRAIN_LABEL2_0 = len(train_data_selected[train_data_selected["2_way_label"] == 0])
TRAIN_LABEL2_1 = len(train_data_selected[train_data_selected["2_way_label"] == 1])
TEST_LABEL2_0 = len(test_data_selected[test_data_selected["2_way_label"] == 0])
TEST_LABEL2_1 = len(test_data_selected[test_data_selected["2_way_label"] == 1])

print(TRAIN_LABEL2_0, TRAIN_LABEL2_1)
print(TEST_LABEL2_0, TEST_LABEL2_1)

341919 222081
35812 23507


In [5]:
TRAIN_LABEL3_0 = len(train_data_selected[train_data_selected["3_way_label"] == 0])
TRAIN_LABEL3_1 = len(train_data_selected[train_data_selected["3_way_label"] == 1])
TRAIN_LABEL3_2 = len(train_data_selected[train_data_selected["3_way_label"] == 2])
TEST_LABEL3_0 = len(test_data_selected[test_data_selected["3_way_label"] == 0])
TEST_LABEL3_1 = len(test_data_selected[test_data_selected["3_way_label"] == 1])
TEST_LABEL3_2 = len(test_data_selected[test_data_selected["3_way_label"] == 2])

print(TRAIN_LABEL3_0, TRAIN_LABEL3_1, TRAIN_LABEL3_2)
print(TEST_LABEL3_0, TEST_LABEL3_1, TEST_LABEL3_2)

222081 13848 328071
23507 1492 34320


In [6]:
TRAIN_LABEL6_0 = len(train_data_selected[train_data_selected["6_way_label"] == 0])
TRAIN_LABEL6_1 = len(train_data_selected[train_data_selected["6_way_label"] == 1])
TRAIN_LABEL6_2 = len(train_data_selected[train_data_selected["6_way_label"] == 2])
TRAIN_LABEL6_3 = len(train_data_selected[train_data_selected["6_way_label"] == 3])
TRAIN_LABEL6_4 = len(train_data_selected[train_data_selected["6_way_label"] == 4])
TRAIN_LABEL6_5 = len(train_data_selected[train_data_selected["6_way_label"] == 5])
TEST_LABEL6_0 = len(test_data_selected[test_data_selected["6_way_label"] == 0])
TEST_LABEL6_1 = len(test_data_selected[test_data_selected["6_way_label"] == 1])
TEST_LABEL6_2 = len(test_data_selected[test_data_selected["6_way_label"] == 2])
TEST_LABEL6_3 = len(test_data_selected[test_data_selected["6_way_label"] == 3])
TEST_LABEL6_4 = len(test_data_selected[test_data_selected["6_way_label"] == 4])
TEST_LABEL6_5 = len(test_data_selected[test_data_selected["6_way_label"] == 5])

print(TRAIN_LABEL6_0, TRAIN_LABEL6_1, TRAIN_LABEL6_2, TRAIN_LABEL6_3, TRAIN_LABEL6_4, TRAIN_LABEL6_5)
print(TEST_LABEL6_0, TEST_LABEL6_1, TEST_LABEL6_2, TEST_LABEL6_3, TEST_LABEL6_4, TEST_LABEL6_5)

222081 33481 107221 11784 167857 21576
23507 3514 11297 1224 17472 2305


In [7]:
ratio_dict = {
    "small" : 0.025,
    "medium" : 0.05,
    "big" : 0.1
}

ratio = ratio_dict[SIZE]

In [8]:
comments_data = pd.read_csv("../data/comments/all_comments.tsv", sep='\t')

/tmp/ipykernel_143161/891078075.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  comments_data = pd.read_csv("../data/comments/all_comments.tsv", sep='\t')


In [9]:
comments_data = comments_data[["body", "submission_id"]]

In [10]:
def search_comments(key):
    comments_list = []
    result = comments_data.loc[comments_data.submission_id == key]
    if not result.empty:
        for idx, row in result.iterrows():
            comments_list.append(row.body)
    return comments_list

#### 2_way_label dataset

In [22]:
top_0_train_data = train_data_selected[train_data_selected["2_way_label"] == 0].head(int(TRAIN_LABEL2_0 * ratio))
top_1_train_data = train_data_selected[train_data_selected["2_way_label"] == 1].head(int(TRAIN_LABEL2_1 * ratio))
train_data_2_classes = pd.concat([top_0_train_data, top_1_train_data])

In [23]:
top_0_test_data = test_data_selected[test_data_selected["2_way_label"] == 0].head(int(TEST_LABEL2_0 * ratio))
top_1_test_data = test_data_selected[test_data_selected["2_way_label"] == 1].head(int(TEST_LABEL2_1 * ratio))
test_data_2_classes = pd.concat([top_0_test_data, top_1_test_data])

In [26]:
train_csv_list = []
title = ["id", "text_comments", "text_only", "comments_only", "2_way_label"]
base_dir = "../data/processed_data/2_classes_small/"

In [27]:
progress_bar = tqdm(train_data_2_classes.iterrows(), total = len(train_data_2_classes))
for idx, row in progress_bar:
    comments_only = ""
    text_comments = ""
    text_comments += row["clean_title"]
    text_comments += "[SEP]"
    comments_list = search_comments(row["id"])
    for each_comment in comments_list:
        comments_only += str(each_comment)
        comments_only += "[SEP]"
        text_comments += str(each_comment)
        text_comments += "[SEP]"
        
    single_list = [row["id"], text_comments, row["clean_title"], comments_only, row["2_way_label"]]
    train_csv_list.append(single_list)

100%|██████████████████████████████████████████████████████████████████| 14099/14099 [1:53:30<00:00,  2.07it/s]


In [28]:
train_csv_file = pd.DataFrame(columns = title, data = train_csv_list)
train_csv_file.to_csv(base_dir + "train_data.csv")

In [29]:
train_csv_file

,id,text_comments,text_only,comments_only,2_way_label
0,98pbid,this concerned sink with a tiny hat[SEP]“Does ...,this concerned sink with a tiny hat,“Does this hat make my head look too big?”[SEP...,0
1,8gnet9,i found a face in my sheet music too[SEP]That ...,i found a face in my sheet music too,That crescendo part at the top also kind of lo...,0
2,6d50rl,major thermos[SEP],major thermos,,0
3,cgp0lmq,cutouts[SEP],cutouts,,0
4,98vrcq,jesus christ converting local teens to christi...,jesus christ converting local teens to christi...,,0
...,...,...,...,...,...
14094,c2vz65,soap dispenser in our classroom looks like lit...,soap dispenser in our classroom looks like lit...,**my classroom doesn’t even have one :D**[SEP],1
14095,ceiuu2,this mint plant in my girlfriends house in lit...,this mint plant in my girlfriends house in lit...,"If you rub your fingers on it, they’ll smell l...",1
14096,5gd0jq,gambia elects a new president for the first ti...,gambia elects a new president for the first ti...,Horrific pop-up ads. Here is an Al Jazeera art...,1
14097,diwhao,i found this rare oreo today with no design on...,i found this rare oreo today with no design on it,I know the inside of an Oreo when I see one![S...,1


In [30]:
test_csv_list = []

In [31]:
progress_bar = tqdm(test_data_2_classes.iterrows(), total = len(test_data_2_classes))
for idx, row in progress_bar:
    comments_only = ""
    text_comments = ""
    text_comments += row["clean_title"]
    text_comments += "[SEP]"
    comments_list = search_comments(row["id"])
    for each_comment in comments_list:
        comments_only += str(each_comment)
        comments_only += "[SEP]"
        text_comments += str(each_comment)
        text_comments += "[SEP]"
        
    single_list = [row["id"], text_comments, row["clean_title"], comments_only, 
                   row["2_way_label"]]
    test_csv_list.append(single_list)

100%|██████████████████████████████████████████████████████████████████████| 1482/1482 [11:47<00:00,  2.10it/s]


In [32]:
test_csv_file = pd.DataFrame(columns = title, data = test_csv_list)
test_csv_file.to_csv(base_dir + "test_data.csv")

In [33]:
test_csv_file

,id,text_comments,text_only,comments_only,2_way_label
0,cozywbv,stargazer[SEP],stargazer,,0
1,ctk61yw,yeah[SEP],yeah,,0
2,7o9rmx,believers hezbollah[SEP]Does Hezbollah actuall...,believers hezbollah,Does Hezbollah actually have kit that nice? Ke...,0
3,bdfxf1,the rise of italian fascism circa[SEP],the rise of italian fascism circa,,0
4,2s0xuj,look at this cool iphone case i got the other ...,look at this cool iphone case i got the other day,Bath bomb?[SEP]I actually saw a iPhone case ev...,0
...,...,...,...,...,...
1477,b7d8lk,i saw a post about being born with fingers her...,i saw a post about being born with fingers her...,You can't fool me. That's a bad guy from Spy K...,1
1478,3ewwi4,sepp blatter deserves nobel prize says vladimi...,sepp blatter deserves nobel prize says vladimi...,"Sure, let's give criminals a prize.[SEP]",1
1479,7sr0x6,reindeer pug[SEP][deleted][SEP][](/user/Grizzl...,reindeer pug,[deleted][SEP][](/user/Grizzlybear911)\n\nPlea...,1
1480,bt27w0,frog left a nearly perfect outline after my do...,frog left a nearly perfect outline after my do...,Dat booty thou.....[SEP]I FIND THIS STINKING A...,1


#### 3_way_label dataset

In [10]:
top_0_train_data = train_data_selected[train_data_selected["3_way_label"] == 0].head(int(TRAIN_LABEL3_0 * ratio))
top_1_train_data = train_data_selected[train_data_selected["3_way_label"] == 1].head(int(TRAIN_LABEL3_1 * ratio * 10))
top_2_train_data = train_data_selected[train_data_selected["3_way_label"] == 2].head(int(TRAIN_LABEL3_2 * ratio))
train_data_3_classes = pd.concat([top_0_train_data, top_1_train_data, top_2_train_data])

In [11]:
top_0_test_data = test_data_selected[test_data_selected["3_way_label"] == 0].head(int(TEST_LABEL3_0 * ratio))
top_1_test_data = test_data_selected[test_data_selected["3_way_label"] == 1].head(int(TEST_LABEL3_1 * ratio * 10))
top_2_test_data = test_data_selected[test_data_selected["3_way_label"] == 2].head(int(TEST_LABEL3_2 * ratio))
test_data_3_classes = pd.concat([top_0_test_data, top_1_test_data, top_2_test_data])

In [12]:
train_csv_list = []

title = ["id", "text_comments", "text_only", "comments_only", "3_way_label"]
base_dir = "../data/processed_data/3_classes_small/"

In [15]:
progress_bar = tqdm(train_data_3_classes.iterrows(), total = len(train_data_3_classes))
for idx, row in progress_bar:
    comments_only = ""
    text_comments = ""
    text_comments += row["clean_title"]
    text_comments += "[SEP]"
    comments_list = search_comments(row["id"])
    for each_comment in comments_list:
        comments_only += str(each_comment)
        comments_only += "[SEP]"
        text_comments += str(each_comment)
        text_comments += "[SEP]"
        
    single_list = [row["id"], text_comments, row["clean_title"], comments_only, row["3_way_label"]]
    train_csv_list.append(single_list)

 79%|████████████████████████████████████████████████████████████▎               | 13666/17215 [1:50:08<27:54,  2.12it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
train_csv_file = pd.DataFrame(columns = title, data = train_csv_list)
train_csv_file.to_csv(base_dir + "train_data.csv")

In [17]:
train_csv_file

,id,text_comments,text_only,comments_only,3_way_label
0,awxhir,my walgreens offbrand mucinex was engraved wit...,my walgreens offbrand mucinex was engraved wit...,Does it help with Dyslexia?[SEP]Pretty sure it...,0
1,6f2cy5,hackers leak emails from uae ambassador to us[...,hackers leak emails from uae ambassador to us,---- **/r/NeutralNews is a curated space.**\nI...,0
2,4xypkv,puppy taking in the view[SEP][EVERYTHING THE L...,puppy taking in the view,[EVERYTHING THE LIGHT TOUCHES IS OUR KINGDOM](...,0
3,2vkbtj,bride and groom exchange vows after fatal shoo...,bride and groom exchange vows after fatal shoo...,"""We planned this weding for more than a year. ...",0
4,86byl8,rabbi meat from cloned pig could be kosher for...,rabbi meat from cloned pig could be kosher for...,Misleading headline: he's saying lab-grown mea...,0
...,...,...,...,...,...
17210,bewghr,xd notorious homosexual[SEP]Me[SEP],xd notorious homosexual,Me[SEP],2
17211,cbszpw9,i think this was the most obvious thing to do[...,i think this was the most obvious thing to do,,2
17212,d992sc5,available here[SEP],available here,,2
17213,d7h9yzm,this is spaaaarrrrttaaaa[SEP],this is spaaaarrrrttaaaa,,2


In [18]:
test_csv_list = []

In [19]:
progress_bar = tqdm(test_data_3_classes.iterrows(), total = len(test_data_3_classes))
for idx, row in progress_bar:
    comments_only = ""
    text_comments = ""
    text_comments += row["clean_title"]
    text_comments += "[SEP]"
    comments_list = search_comments(row["id"])
    for each_comment in comments_list:
        comments_only += str(each_comment)
        comments_only += "[SEP]"
        text_comments += str(each_comment)
        text_comments += "[SEP]"
        
    single_list = [row["id"], text_comments, row["clean_title"], comments_only, 
                   row["3_way_label"]]
    test_csv_list.append(single_list)

100%|████████████████████████████████████████████████████████████████████████████████| 1818/1818 [14:28<00:00,  2.09it/s]


In [20]:
test_csv_file = pd.DataFrame(columns = title, data = test_csv_list)
test_csv_file.to_csv(base_dir + "test_data.csv")

In [21]:
test_csv_file

,id,text_comments,text_only,comments_only,3_way_label
0,c0gl7r,pd phoenix car thief gets instructions from yo...,pd phoenix car thief gets instructions from yo...,> Police say they found a phone playing a YouT...,0
1,c0xdqy,as trump accuses iran he has one problem his o...,as trump accuses iran he has one problem his o...,---- **/r/NeutralNews is a curated space.**\nI...,0
2,8g3xtm,trumps pick to lead ice who touted surge in im...,trumps pick to lead ice who touted surge in im...,---- **/r/NeutralNews is a curated space.**\nI...,0
3,26kwdf,my friend midflip[SEP][Skewered](http://imgur....,my friend midflip,[Skewered](http://imgur.com/Ju1TALf)[SEP][We h...,0
4,bopb6s,this smiling dog[SEP]Why did I do this \n\nhtt...,this smiling dog,Why did I do this \n\nhttps://ibb.co/ss7kG8Q[S...,0
...,...,...,...,...,...
1813,cx0pag8,its a trap[SEP],its a trap,,2
1814,76j65g,special routes of us route marvellous super hu...,special routes of us route marvellous super hu...,,2
1815,cuuofh0,mmmpact[SEP],mmmpact,,2
1816,38u6eo,screaming shoes[SEP]:D :D[SEP],screaming shoes,:D :D[SEP],2


#### 6_way_label dataset

In [11]:
top_0_train_data = train_data_selected[train_data_selected["6_way_label"] == 0].head(int(TRAIN_LABEL6_0 * ratio))
top_1_train_data = train_data_selected[train_data_selected["6_way_label"] == 1].head(int(TRAIN_LABEL6_1 * ratio * 10))
top_2_train_data = train_data_selected[train_data_selected["6_way_label"] == 2].head(int(TRAIN_LABEL6_2 * ratio))
top_3_train_data = train_data_selected[train_data_selected["6_way_label"] == 3].head(int(TRAIN_LABEL6_3 * ratio * 10))
top_4_train_data = train_data_selected[train_data_selected["6_way_label"] == 4].head(int(TRAIN_LABEL6_4 * ratio))
top_5_train_data = train_data_selected[train_data_selected["6_way_label"] == 5].head(int(TRAIN_LABEL6_5 * ratio * 10))
train_data_6_classes = pd.concat([top_0_train_data, top_1_train_data, top_2_train_data, top_3_train_data, top_4_train_data, top_5_train_data])

In [12]:
top_0_test_data = test_data_selected[test_data_selected["6_way_label"] == 0].head(int(TEST_LABEL6_0 * ratio))
top_1_test_data = test_data_selected[test_data_selected["6_way_label"] == 1].head(int(TEST_LABEL6_1 * ratio * 10))
top_2_test_data = test_data_selected[test_data_selected["6_way_label"] == 2].head(int(TEST_LABEL6_2 * ratio))
top_3_test_data = test_data_selected[test_data_selected["6_way_label"] == 3].head(int(TEST_LABEL6_3 * ratio * 10))
top_4_test_data = test_data_selected[test_data_selected["6_way_label"] == 4].head(int(TEST_LABEL6_4 * ratio))
top_5_test_data = test_data_selected[test_data_selected["6_way_label"] == 5].head(int(TEST_LABEL6_5 * ratio * 10))
test_data_6_classes = pd.concat([top_0_test_data, top_1_test_data, top_2_test_data, top_3_test_data, top_4_test_data, top_5_test_data])

In [14]:
train_csv_list = []

title = ["id", "text_comments", "text_only", "comments_only", "6_way_label"]
base_dir = "../data/processed_data/6_classes_small/"

In [15]:
progress_bar = tqdm(train_data_6_classes.iterrows(), total = len(train_data_6_classes))
for idx, row in progress_bar:
    comments_only = ""
    text_comments = ""
    text_comments += row["clean_title"]
    text_comments += "[SEP]"
    comments_list = search_comments(row["id"])
    for each_comment in comments_list:
        comments_only += str(each_comment)
        comments_only += "[SEP]"
        text_comments += str(each_comment)
        text_comments += "[SEP]"
        
    single_list = [row["id"], text_comments, row["clean_title"], comments_only, row["6_way_label"]]
    train_csv_list.append(single_list)

 59%|█████████████████████████████████████████████▋                                | 17065/29138 [2:19:20<1:38:35,  2.04it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|████████████████████████████████████████████████████████████████████████████████| 29138/29138 [3:55:43<00:00,  2.06it/s]


In [17]:
train_csv_file = pd.DataFrame(columns = title, data = train_csv_list)
train_csv_file.to_csv(base_dir + "train_data.csv")

In [18]:
train_csv_file

,id,text_comments,text_only,comments_only,6_way_label
0,awxhir,my walgreens offbrand mucinex was engraved wit...,my walgreens offbrand mucinex was engraved wit...,Does it help with Dyslexia?[SEP]Pretty sure it...,0
1,6f2cy5,hackers leak emails from uae ambassador to us[...,hackers leak emails from uae ambassador to us,---- **/r/NeutralNews is a curated space.**\nI...,0
2,4xypkv,puppy taking in the view[SEP][EVERYTHING THE L...,puppy taking in the view,[EVERYTHING THE LIGHT TOUCHES IS OUR KINGDOM](...,0
3,2vkbtj,bride and groom exchange vows after fatal shoo...,bride and groom exchange vows after fatal shoo...,"""We planned this weding for more than a year. ...",0
4,86byl8,rabbi meat from cloned pig could be kosher for...,rabbi meat from cloned pig could be kosher for...,Misleading headline: he's saying lab-grown mea...,0
...,...,...,...,...,...
29133,2jurut,better together celtic workers party modern[SE...,better together celtic workers party modern,This is a poster made for the Celtic Worker's ...,5
29134,7tguct,is this google maps sighting proof that aliens...,is this google maps sighting proof that aliens...,A lake that rotates? Sounds like some alien te...,5
29135,biozfe,ussr presumably s its says un united nations o...,ussr presumably s its says un united nations o...,You have to give credit to ussr propoganda dep...,5
29136,3utxy3,a collection of posters for you all[SEP]I had ...,a collection of posters for you all,I had previously shared this album on an old a...,5


In [19]:
test_csv_list = []

In [20]:
progress_bar = tqdm(test_data_6_classes.iterrows(), total = len(test_data_6_classes))
for idx, row in progress_bar:
    comments_only = ""
    text_comments = ""
    text_comments += row["clean_title"]
    text_comments += "[SEP]"
    comments_list = search_comments(row["id"])
    for each_comment in comments_list:
        comments_only += str(each_comment)
        comments_only += "[SEP]"
        text_comments += str(each_comment)
        text_comments += "[SEP]"
        
    single_list = [row["id"], text_comments, row["clean_title"], comments_only, 
                   row["6_way_label"]]
    test_csv_list.append(single_list)

100%|████████████████████████████████████████████████████████████████████████████████| 3065/3065 [25:00<00:00,  2.04it/s]


In [21]:
test_csv_file = pd.DataFrame(columns = title, data = test_csv_list)
test_csv_file.to_csv(base_dir + "test_data.csv")

In [22]:
test_csv_file

,id,text_comments,text_only,comments_only,6_way_label
0,c0gl7r,pd phoenix car thief gets instructions from yo...,pd phoenix car thief gets instructions from yo...,> Police say they found a phone playing a YouT...,0
1,c0xdqy,as trump accuses iran he has one problem his o...,as trump accuses iran he has one problem his o...,---- **/r/NeutralNews is a curated space.**\nI...,0
2,8g3xtm,trumps pick to lead ice who touted surge in im...,trumps pick to lead ice who touted surge in im...,---- **/r/NeutralNews is a curated space.**\nI...,0
3,26kwdf,my friend midflip[SEP][Skewered](http://imgur....,my friend midflip,[Skewered](http://imgur.com/Ju1TALf)[SEP][We h...,0
4,bopb6s,this smiling dog[SEP]Why did I do this \n\nhtt...,this smiling dog,Why did I do this \n\nhttps://ibb.co/ss7kG8Q[S...,0
...,...,...,...,...,...
3060,38thux,nixon cuba[SEP]What's interesting about Cuba's...,nixon cuba,What's interesting about Cuba's brand of propa...,5
3061,62glwz,stop them[SEP]That caricature looks a bit too ...,stop them,That caricature looks a bit too antisemitic fo...,5
3062,6sehpd,girl deeply outraged by this sweater should ta...,girl deeply outraged by this sweater should ta...,"Better than the ""I'm Retarded for X-mas"" sweat...",5
3063,4l8ul9,what happened when an alreadycrazed city charg...,what happened when an alreadycrazed city charg...,"""Hi there! Would you sign my petition?""[SEP]Th...",5
